In [11]:
from pyspark.sql import SparkSession

spark = (
    
    SparkSession
            .builder
            .config('spark.jars.packages', 'io.delta:delta-core_2.12:2.1.0')
            .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
            .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
            .getOrCreate()
            
)

In [12]:
sc = spark.sparkContext

In [15]:
lotr = (
    spark
    .read
    .format('delta')
    .load('../datalake/silver/lotr_pt_br')
)

In [14]:
import pyspark.sql.functions as F

In [17]:
agg_advices = (
    lotr
    .withColumn('word', F.explode(F.split(F.col('personagem'), ' ')))
    .groupBy('word')
    .count()
    .sort('count', ascending=False)
)

In [18]:
agg_advices.show()

+-------+-----+
|   word|count|
+-------+-----+
|  FRODO|   49|
|    SAM|   48|
|GANDALF|   40|
| PIPPIN|   30|
|SMEAGOL|   24|
|ARAGORN|   20|
|THEODEN|   20|
|  GIMLI|   16|
|SARUMAN|   16|
|    ORC|   14|
|  MERRY|   14|
|  EOWYN|   12|
|   KING|   10|
|  BILBO|   10|
|  WITCH|   10|
| GOLLUM|   10|
| SAURON|    8|
|     OF|    8|
|  MOUTH|    8|
| DEAGOL|    6|
+-------+-----+
only showing top 20 rows



In [19]:
(
    agg_advices
    .write
    .format('delta')
    .mode('overwrite')
    .save('../datalake/gold/agg_lotr')
)